In [ ]:
import numpy as np
import pandas as pd
import json


from pandas.io.json import json_normalize

In [ ]:
movies = pd.read_csv('movies_metadata.csv')

In [3]:
# checking shape 
movies.shape

(45466, 24)

In [4]:
# checking some first rows:

movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [5]:
# checking nan values
movies.isnull().sum()/movies.shape[0] 

adult                    0.000000
belongs_to_collection    0.901157
budget                   0.000000
genres                   0.000000
homepage                 0.828839
id                       0.000000
imdb_id                  0.000374
original_language        0.000242
original_title           0.000000
overview                 0.020983
popularity               0.000110
poster_path              0.008490
production_companies     0.000066
production_countries     0.000066
release_date             0.001914
revenue                  0.000132
runtime                  0.005785
spoken_languages         0.000132
status                   0.001914
tagline                  0.551049
title                    0.000132
video                    0.000132
vote_average             0.000132
vote_count               0.000132
dtype: float64

In [6]:
# get the percentage of the nan value for each attribute
null_cols = movies.isnull().sum()/movies.shape[0]
# get the attribute which has more than 50% are nan values
null_cols[null_cols>0.5] 

belongs_to_collection    0.901157
homepage                 0.828839
tagline                  0.551049
dtype: float64

In [7]:
# Dropping the attributes which have more than 50 missing values
movies.drop(columns = list(null_cols[null_cols > 0.5].index), axis =1, inplace = True)

In [8]:
# Check if the attributes are dropped
movies.shape

(45466, 21)

In [10]:
# dropping not important columns
movies.drop(columns=['overview','poster_path','status'], axis =1, inplace =True)


In [11]:
movies.isnull().sum()

adult                     0
budget                    0
genres                    0
id                        0
imdb_id                  17
original_language        11
original_title            0
popularity                5
production_companies      3
production_countries      3
release_date             87
revenue                   6
runtime                 263
spoken_languages          6
title                     6
video                     6
vote_average              6
vote_count                6
dtype: int64

In [12]:
# drop all rows with nan values, since they are only 300 rows / 45466 rows in the original dataframe
#reset index after drop, use the drop to avoid the old index being added as a column
movies = movies.dropna().reset_index(drop=True) 


In [13]:
movies = movies.rename(columns={"id": "movie_id"}) 
#Renamed beacause we have id in side the crew and the cast columns, will use it to merge dataframes


## GENRES: 
#### - the data in Genres column is a json inside a list 
#### - See file Genres.ipynb for more details


In [15]:
genres_1 = []
for i,r in enumerate(movies.genres): 
    j = eval(r)
    for k in j: #j is now one dictionary inside the list, we want to add the movies id into each dictionary
        k['movie_id'] = movies['movie_id'][i]
    genres_1.append(j)
    
genres=[]
for i in genres_1:
    genres.extend(i)  #use extend instead of append because we want only 1 list, not 1ists inside the list

genres = json_normalize(genres)  # flatten all the json in the list
genres.to_csv('Genres.csv',index=False) # saving to csv for future usage



##  Production_companie
#### - See file Productions.ipynb for more details

In [16]:
prod_1 = []
for i,r in enumerate(movies.production_companies): 
    j = eval(r)
    for k in j: 
        k['movie_id'] = movies['movie_id'][i]
    prod_1.append(j)

prod=[]
for i in prod_1:
    prod.extend(i)  #use extend instead of append because we want only 1 list, not 1ists inside the list

prod = json_normalize(prod)
prod.to_csv('Productions.csv',index=False)

## Countries
#### - See file Countries.ipynb for more details

In [17]:
# GETTING the production_companies:
country_1 = []
for i,r in enumerate(movies.production_countries): 
    j = eval(r)
    for k in j:    
        k['movie_id'] = movies['movie_id'][i]
    country_1.append(j)

country=[]
for i in prod_1:
    country.extend(i)  #use extend instead of append because we want only 1 list, not 1ists inside the list

country = json_normalize(country)
country.to_csv('Countries.csv',index=False)

## SCRAPING IMDB FOR REVENUE and BUDGET
#### - because most of the budget and revenue are == 0

In [22]:
movies.revenue.value_counts()

0.0            37706
12000000.0        20
11000000.0        19
10000000.0        19
2000000.0         18
6000000.0         17
5000000.0         14
500000.0          13
8000000.0         13
1.0               12
14000000.0        12
7000000.0         11
3000000.0         10
1000000.0         10
20000000.0        10
1500000.0          9
4000000.0          9
16000000.0         8
30000000.0         8
4100000.0          8
25000000.0         8
15000000.0         8
2500000.0          8
3.0                8
18000000.0         7
4300000.0          7
13000000.0         7
1400000.0          6
100000000.0        6
9000000.0          6
               ...  
26533200.0         1
40485039.0         1
122915111.0        1
80936232.0         1
161849455.0        1
245676146.0        1
18130888.0         1
316000.0           1
29450919.0         1
6629178.0          1
89289910.0         1
1133031.0          1
20225989.0         1
40452643.0         1
26518355.0         1
57231524.0         1
10113733.0   

In [23]:
movies.budget.value_counts()

0            36232
5000000        286
10000000       259
20000000       243
2000000        241
15000000       226
3000000        223
25000000       206
1000000        195
30000000       190
4000000        181
6000000        173
12000000       172
40000000       167
8000000        155
500000         142
35000000       141
7000000        137
50000000       124
1500000        120
60000000       110
18000000       105
3500000        103
2500000         98
13000000        89
9000000         87
11000000        83
14000000        76
16000000        75
22000000        71
             ...  
598500           1
8620000          1
280000000        1
1282545          1
3730500          1
46630000         1
10200000         1
2640000          1
220000000        1
5904067          1
5860000          1
110125           1
525000           1
3774400          1
6153120          1
2535000          1
884130           1
1814462          1
7300             1
1520000          1
230              1
3383700     

In [ ]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def webscrape(url):
    import re
    html = r.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    element = soup.get_text()
    revenue = re.findall(r"Cumulative Worldwide Gross: (.*)        " ,element)
    budget = re.findall(r"Budget:(.*)" ,element)
    if len(revenue)==0:  # to avoide error when there is no info about revenue or budget
        revenue.append('$0')
    if len(budget)==0:
        budget.append('$0')
    return revenue[0],budget[0] # pick the 1st element from the revenue and budget list, 
#resulting only the element, not inside the list


box_office={}
count=0
for i in movies.imdb_id:
    url = f'https://www.imdb.com/title/{i}/'
    scrape = webscrape(url)
    box_office.update({i:scrape})
    count+=1
    if count%100==0:
        print(i,count)
    #revenue[i]=revenue[scrape]
    #revenue1.append(revenue)
    #revenue.extend(scrape)


#Putting the revenue and budget into dataframe (bo)
bo = pd.DataFrame(box_office).T
bo.reset_index(inplace=True)
bo.to_csv('box_office.csv')

# Convert release_date into Date Month Year 

In [25]:
import datetime

In [26]:
from datetime import date

In [27]:
movies['release_date1']= pd.to_datetime(movies['release_date'].astype(str))

In [28]:
movies['Month']=movies['release_date1'].dt.month

In [29]:
movies['Year']=movies['release_date1'].dt.month

In [47]:
movies['Month']

0        10
1        12
2        12
3        12
4         2
5        12
6        12
7        12
8        12
9        11
10       11
11       12
12       12
13       12
14       12
15       11
16       12
17       12
18       11
19       11
20       10
21       10
22       10
23       10
24       10
25       12
26       10
27        9
28        5
29        4
         ..
44319    10
44320     3
44321     1
44322     6
44323     4
44324     6
44325     1
44326     3
44327     1
44328     1
44329     1
44330     1
44331     3
44332     1
44333     9
44334     1
44335     1
44336     1
44337     7
44338     1
44339     1
44340     3
44341    10
44342    10
44343     1
44344     5
44345    11
44346     8
44347    10
44348     6
Name: Month, Length: 44349, dtype: int64